In [15]:
############# Loading ##############

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import pickle

# Regressors
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from xgboost.sklearn import XGBRegressor 
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
import lightgbm as lgb

# Evaluation metrics
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_percentage_error as mape

epsoln = 1e-9  # Define a small epsilon value for division by zero cases

def rmse(y_true, y_pred):
  return np.sqrt(mse(y_true, y_pred))

def pbe(y_true, y_pred):
  if np.sum(y_true)!=0:
    return 100*(np.sum(y_pred - y_true)/np.sum(y_true))
  else:
    return 100*(np.sum(y_pred - y_true)/(np.sum(y_true)+epsoln))

def pocid(y_true, y_pred):
  n = len(y_true)
  D = [1 if (y_pred[i] - y_pred[i-1]) * (y_true[i] - y_true[i-1]) > 0 else 0 for i in range(1, n)]
  POCID = 100 * np.sum(D) / (n-1)
  return POCID

def mcpm(rmse_result, mape_result, pocid_result):
  er_result = 100 - pocid_result

  A1 = (rmse_result * mape_result * np.sin((2*np.pi)/3))/2
  A2 = (mape_result * er_result * np.sin((2*np.pi)/3))/2
  A3 = (er_result * rmse_result * np.sin((2*np.pi)/3))/2
  total = A1 + A2 + A3
  return total

def znorm(x):
  if np.std(x) != 0: 
      x_znorm = (x - np.mean(x)) / np.std(x)
  else:
      x_znorm = (x - np.mean(x)) / (np.std(x) + epsoln)
  return x_znorm

def znorm_reverse(x, mean_x, std_x):
  x_denormalized = (np.array(x) * std_x) + mean_x
  return x_denormalized

def get_stats_norm(series, horizon, window):
  last_subsequence = series[-(horizon+window):-horizon].values
  last_mean = np.mean(last_subsequence)
  last_std = np.std(last_subsequence)
  return last_mean, last_std

# Em geral, considera-se um tamanho de janela capaz de capturar um ciclo dos dados
# Por exemplo, 12 observações no caso dos dados com frequência mensal
def rolling_window(series, window):
  data = []
  for i in range(len(series)-window):
    example = znorm(np.array(series[i:i+window+1]))
    data.append(example)
  df = pd.DataFrame(data)
  return df

# Para predição de vendas por UF (mensal), será considerado horizon = 12
# Para predição de vendas por município (anual), será considerado horizon = 1
def train_test_split(data, horizon):
  X = data.iloc[:,:-1] # features
  y = data.iloc[:,-1] # target

  X_train = X[:-horizon] # features train
  X_test =  X[-horizon:] # features test

  y_train = y[:-horizon] # target train
  y_test = y[-horizon:] # target test
  return X_train, X_test, y_train, y_test

def recursive_multistep_forecasting(X_test, model, horizon):
  # example é composto pelas últimas observações vistas
  # na prática, é o pbeprimeiro exemplo do conjunto de teste
  example = X_test.iloc[0].values.reshape(1,-1)

  preds = []
  for i in range(horizon):
    pred = model.predict(example)[0]
    preds.append(pred)

    # Descartar o valor da primeira posição do vetor de características
    example = example[:,1:]

    # Adicionar o valor predito na última posição do vetor de características
    example = np.append(example, pred)
    example = example.reshape(1,-1)
  return preds

def baseline_mean(series, horizon):
  # como as séries são normalizadas, esse baseline irá retornar uma reta próxima de zero
  pred = np.repeat(np.mean(znorm(series[:-horizon])), horizon)
  return pred

def baseline_persistent(series, horizon):
  return np.repeat(znorm(series[-2*horizon:-horizon]).values[-1], horizon)

def baseline_persistent_window(series, horizon):
  subsequence = znorm(series[-horizon*2:-horizon]).values
  return subsequence


In [16]:
######### Lista Estados ##############

import os
import csv

def extract_estado(file_name):
    parts = file_name.split('_')
    estado = parts[1]
    return estado

def read_csv_files(folder_path):
    estados = []
    files = os.listdir(folder_path)
    for file_name in files:
        if file_name.endswith('.csv'):
            file_path = os.path.join(folder_path, file_name)
            with open(file_path, 'r', newline='') as csvfile:
                reader = csv.reader(csvfile)
                headers = next(reader)
                estado = extract_estado(file_name)
                estados.append(estado)
                estados.sort()
    return estados




In [14]:
######### Regressors TEST #######################

products = sorted([name for name in os.listdir('./uf') if os.path.isdir(os.path.join('./uf', name))])

# Parâmetros
# - horizon: horizonte de predição >> Padrão 12
# - window: comprimento da janela para a geração da tabela atributo-valor >> Padrão 12
horizon = 12
window = 12

#####################################
product = 'etanolhidratado'
estado = 'ac'
#####################################

# for product in products:
#     folder_path = f'./uf/{product}/'
#     # Read the CSV files and extract estado names
#     estados = read_csv_files(folder_path)
#     for estado in estados:

# carregamento do arquivo
df = pd.read_csv(f"./uf/{product}/mensal_{estado}_{product}.csv", header=0, sep=";")
series = df["m3"]

# Geração da tabela atributo-valor (normalizado)
data = rolling_window(series, window)

# Divisão dos dados em treino/teste considerando o horizonte de predição de 12 meses
X_train, X_test, y_train, y_test = train_test_split(data, horizon)

############## Regressores ##################
regr1 = LinearRegression()
regr2 = KNeighborsRegressor(n_neighbors = 3)
regr3 = XGBRegressor()
regr4 = SVR(kernel='rbf')
regr5 = RandomForestRegressor()
regr6 = MLPRegressor(random_state=1, activation='relu', max_iter=500)

# Treinamento dos modelos
regr1.fit(X_train, y_train)
predictions1a = recursive_multistep_forecasting(X_test, regr1, horizon)
regr2.fit(X_train, y_train)
predictions2a = recursive_multistep_forecasting(X_test, regr2, horizon)
regr3.fit(X_train, y_train)
predictions3a = recursive_multistep_forecasting(X_test, regr3, horizon)
regr4.fit(X_train, y_train)
predictions4a = recursive_multistep_forecasting(X_test, regr4, horizon)
regr5.fit(X_train, y_train)
predictions5a = recursive_multistep_forecasting(X_test, regr5, horizon)
regr6.fit(X_train, y_train)
predictions6a = recursive_multistep_forecasting(X_test, regr6, horizon)

# Recupera a média e desvio-padrão da última subsequência observada
mean_norm, std_norm = get_stats_norm(series, horizon, window)

# Reescala a predição
predictions1 = znorm_reverse(predictions1a, mean_norm, std_norm)
predictions2 = znorm_reverse(predictions2a, mean_norm, std_norm)
predictions3 = znorm_reverse(predictions3a, mean_norm, std_norm)
predictions4 = znorm_reverse(predictions4a, mean_norm, std_norm)
predictions5 = znorm_reverse(predictions5a, mean_norm, std_norm)
predictions6 = znorm_reverse(predictions6a, mean_norm, std_norm)

Valores_Reais = series.tail(12)
Valores_Reais = Valores_Reais.reset_index(drop=True)

mape_result1 = mape(Valores_Reais, predictions1)
mape_result2 = mape(Valores_Reais, predictions2)
mape_result3 = mape(Valores_Reais, predictions3)
mape_result4 = mape(Valores_Reais, predictions4)
mape_result5 = mape(Valores_Reais, predictions5)
mape_result6 = mape(Valores_Reais, predictions6)

pbe_result1 = pbe(Valores_Reais, predictions1)
pbe_result2 = pbe(Valores_Reais, predictions2)
pbe_result3 = pbe(Valores_Reais, predictions3)
pbe_result4 = pbe(Valores_Reais, predictions4)
pbe_result5 = pbe(Valores_Reais, predictions5)
pbe_result6 = pbe(Valores_Reais, predictions6)

pocid_result1 = pocid(Valores_Reais, predictions1)
pocid_result2 = pocid(Valores_Reais, predictions2)
pocid_result3 = pocid(Valores_Reais, predictions3)
pocid_result4 = pocid(Valores_Reais, predictions4)
pocid_result5 = pocid(Valores_Reais, predictions5)
pocid_result6 = pocid(Valores_Reais, predictions6)

predictions1b = ', '.join(f"{item:f}" for item in predictions1)
predictions2b = ', '.join(f"{item:f}" for item in predictions2)
predictions3b = ', '.join(f"{item:f}" for item in predictions3)
predictions4b = ', '.join(f"{item:f}" for item in predictions4)
predictions5b = ', '.join(f"{item:f}" for item in predictions5)
predictions6b = ', '.join(f"{item:f}" for item in predictions6)

rows_data = [
    [product,estado,'LR',mape_result1,pocid_result1,pbe_result1,predictions1b],
    [product,estado,'kNN',mape_result2,pocid_result2,pbe_result2,predictions2b],
    [product,estado,'XGB',mape_result3,pocid_result3,pbe_result3,predictions3b],
    [product,estado,'SVR',mape_result4,pocid_result4,pbe_result4,predictions4b],
    [product,estado,'RF',mape_result5,pocid_result5,pbe_result5,predictions5b],
    [product,estado,'MLP',mape_result6,pocid_result6,pbe_result6,predictions6b],
]

# Salva Modelos
folder_path = f"./00-MODELS_UF_MENSAL-RawData/"
os.makedirs(folder_path, exist_ok=True)

with open(os.path.join(folder_path, f'RawData_LR_{product}_{estado}_{window}_model.pkl'), 'wb') as fd1:
    pickle.dump(regr2, fd1)
with open(os.path.join(folder_path, f'RawData_KNN_{product}_{estado}_{window}_model.pkl'), 'wb') as fd2:
    pickle.dump(regr2, fd2)
with open(os.path.join(folder_path, f'RawData_XGB_{product}_{estado}_{window}_model.pkl'), 'wb') as fd3:
    pickle.dump(regr3, fd3)
with open(os.path.join(folder_path, f'RawData_SVR_{product}_{estado}_{window}_model.pkl'), 'wb') as fd4:
    pickle.dump(regr4, fd4)
with open(os.path.join(folder_path, f'RawData_RF_{product}_{estado}_{window}_model.pkl'), 'wb') as fd5:
    pickle.dump(regr5, fd5)
with open(os.path.join(folder_path, f'RawData_MLP_{product}_{estado}_{window}_model.pkl'), 'wb') as fd6:
    pickle.dump(regr2, fd6)


# CSV Output VALORES REAIS
with open(f'RawData_{window}_norm_output.csv', 'a', newline='') as file:
    writer = csv.writer(file)
    for row_data in rows_data:
        writer.writerow(row_data)

In [17]:
######### Regressors AUTO #######################

products = sorted([name for name in os.listdir('./uf') if os.path.isdir(os.path.join('./uf', name))])

# Parâmetros
# - horizon: horizonte de predição >> Padrão 12
# - window: comprimento da janela para a geração da tabela atributo-valor >> Padrão 12
horizon = 12
window = 12

for product in products:
    folder_path = f'./uf/{product}/'
    # Read the CSV files and extract estado names
    estados = read_csv_files(folder_path)
    
    for estado in estados:
    
        # carregamento do arquivo
        df = pd.read_csv(f"./uf/{product}/mensal_{estado}_{product}.csv", header=0, sep=";")
    
        series = df["m3"]
   
        # Geração da tabela atributo-valor (normalizado)
        data = rolling_window(series, window)
    
        # Divisão dos dados em treino/teste considerando o horizonte de predição de 12 meses
        X_train, X_test, y_train, y_test = train_test_split(data, horizon)
    
        ############## Regressores ##################
        regr1 = LinearRegression()
        regr2 = KNeighborsRegressor(n_neighbors = 3)
        regr3 = XGBRegressor()
        regr4 = SVR(kernel='rbf')
        regr5 = RandomForestRegressor()
        regr6 = MLPRegressor(random_state=1, activation='relu', max_iter=500)

        # Treinamento dos modelos
        regr1.fit(X_train, y_train)
        predictions1a = recursive_multistep_forecasting(X_test, regr1, horizon)
        regr2.fit(X_train, y_train)
        predictions2a = recursive_multistep_forecasting(X_test, regr2, horizon)
        regr3.fit(X_train, y_train)
        predictions3a = recursive_multistep_forecasting(X_test, regr3, horizon)
        regr4.fit(X_train, y_train)
        predictions4a = recursive_multistep_forecasting(X_test, regr4, horizon)
        regr5.fit(X_train, y_train)
        predictions5a = recursive_multistep_forecasting(X_test, regr5, horizon)
        regr6.fit(X_train, y_train)
        predictions6a = recursive_multistep_forecasting(X_test, regr6, horizon)

        # Recupera a média e desvio-padrão da última subsequência observada
        mean_norm, std_norm = get_stats_norm(series, horizon, window)

        # Reescala a predição
        predictions1 = znorm_reverse(predictions1a, mean_norm, std_norm)
        predictions2 = znorm_reverse(predictions2a, mean_norm, std_norm)
        predictions3 = znorm_reverse(predictions3a, mean_norm, std_norm)
        predictions4 = znorm_reverse(predictions4a, mean_norm, std_norm)
        predictions5 = znorm_reverse(predictions5a, mean_norm, std_norm)
        predictions6 = znorm_reverse(predictions6a, mean_norm, std_norm)

        Valores_Reais = series.tail(12)
        Valores_Reais = Valores_Reais.reset_index(drop=True)

        mape_result1 = mape(Valores_Reais, predictions1)
        mape_result2 = mape(Valores_Reais, predictions2)
        mape_result3 = mape(Valores_Reais, predictions3)
        mape_result4 = mape(Valores_Reais, predictions4)
        mape_result5 = mape(Valores_Reais, predictions5)
        mape_result6 = mape(Valores_Reais, predictions6)

        pbe_result1 = pbe(Valores_Reais, predictions1)
        pbe_result2 = pbe(Valores_Reais, predictions2)
        pbe_result3 = pbe(Valores_Reais, predictions3)
        pbe_result4 = pbe(Valores_Reais, predictions4)
        pbe_result5 = pbe(Valores_Reais, predictions5)
        pbe_result6 = pbe(Valores_Reais, predictions6)

        pocid_result1 = pocid(Valores_Reais, predictions1)
        pocid_result2 = pocid(Valores_Reais, predictions2)
        pocid_result3 = pocid(Valores_Reais, predictions3)
        pocid_result4 = pocid(Valores_Reais, predictions4)
        pocid_result5 = pocid(Valores_Reais, predictions5)
        pocid_result6 = pocid(Valores_Reais, predictions6)

        predictions1b = ', '.join(f"{item:f}" for item in predictions1)
        predictions2b = ', '.join(f"{item:f}" for item in predictions2)
        predictions3b = ', '.join(f"{item:f}" for item in predictions3)
        predictions4b = ', '.join(f"{item:f}" for item in predictions4)
        predictions5b = ', '.join(f"{item:f}" for item in predictions5)
        predictions6b = ', '.join(f"{item:f}" for item in predictions6)

        rows_data = [
            [product,estado,'LR',mape_result1,pocid_result1,pbe_result1,predictions1b],
            [product,estado,'kNN',mape_result2,pocid_result2,pbe_result2,predictions2b],
            [product,estado,'XGB',mape_result3,pocid_result3,pbe_result3,predictions3b],
            [product,estado,'SVR',mape_result4,pocid_result4,pbe_result4,predictions4b],
            [product,estado,'RF',mape_result5,pocid_result5,pbe_result5,predictions5b],
            [product,estado,'MLP',mape_result6,pocid_result6,pbe_result6,predictions6b],
        ]

        # Salva Modelos
        folder_path = f"./00-MODELS_UF_MENSAL-RawData/"
        os.makedirs(folder_path, exist_ok=True)

        with open(os.path.join(folder_path, f'RawData_LR_{product}_{estado}_{window}_model.pkl'), 'wb') as fd1:
            pickle.dump(regr2, fd1)
        with open(os.path.join(folder_path, f'RawData_KNN_{product}_{estado}_{window}_model.pkl'), 'wb') as fd2:
            pickle.dump(regr2, fd2)
        with open(os.path.join(folder_path, f'RawData_XGB_{product}_{estado}_{window}_model.pkl'), 'wb') as fd3:
            pickle.dump(regr3, fd3)
        with open(os.path.join(folder_path, f'RawData_SVR_{product}_{estado}_{window}_model.pkl'), 'wb') as fd4:
            pickle.dump(regr4, fd4)
        with open(os.path.join(folder_path, f'RawData_RF_{product}_{estado}_{window}_model.pkl'), 'wb') as fd5:
            pickle.dump(regr5, fd5)
        with open(os.path.join(folder_path, f'RawData_MLP_{product}_{estado}_{window}_model.pkl'), 'wb') as fd6:
            pickle.dump(regr2, fd6)


        # CSV Output VALORES REAIS
        with open(f'RawData_{window}_norm_output.csv', 'a', newline='') as file:
            writer = csv.writer(file)
            for row_data in rows_data:
                writer.writerow(row_data)